In [2]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/25 11:15:30 WARN Utils: Your hostname, LAPTOP-MMSL4U2D resolves to a loopback address: 127.0.1.1; using 172.18.73.152 instead (on interface eth0)
24/09/25 11:15:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 11:15:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
trans_final_sdf = spark.read.parquet('../data/merged/merged_transactions_with_outliers.parquet')
trans_final_sdf.show(5)

+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+-----------+-----+--------+------+--------------------------+--------------------------+--------------+--------------------+------------------+------------------+----------------------+--------------------+------------+--------------------+--------------------------+--------------------+--------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|       merchant_name|                tags|consumer_id|state|postcode|gender|consumer_fraud_probability|merchant_fraud_probability| LOCALITY_NAME|       list_SA2_CODE|          list_ERP|  list_num_earners|list_total_income_($M)|    list_mean_income|postcode_ERP|postcode_num_earners|postcode_total_income_($M)|postcode_mean_income|is_fraud|
+-------+------------+------------------+--------------------+--------------+--------------------+--------------------+-----------+-----+--------+------+-------------

In [4]:
# Function to convert to string of list
import re
def process_string(input_string):
    # Replace () with []
    input_string = input_string.replace('(', '[')
    input_string = input_string.replace(')', ']')

    # Use regex to split by outer brackets and commas
    lists = re.findall(r'\[([^\[\]]+)\]', input_string)

    # Further split elements within the lists based on commas, while stripping whitespace
    result = [list(map(str.strip, item.split(','))) for item in lists]

    # Clean up each item by stripping leading/trailing whitespace and correcting case
    for sublist in result:
        for i in range(len(sublist)):
            sublist[i] = sublist[i].strip().lower()

    return result
# Register UDF in PySpark
process_string_udf = F.udf(process_string)

# Apply the UDF to the DataFrame
result_df = trans_final_sdf.withColumn("fixed_tags", process_string_udf(F.col("tags")))

# Show the result
result_df.show(5, truncate=False)

+-------+------------+------------------+------------------------------------+--------------+------------------------------+-------------------------------------------------------------------------+-----------+-----+--------+------+--------------------------+--------------------------+--------------+---------------------------------+------------------+------------------+----------------------+---------------------------+------------+--------------------+--------------------------+--------------------+--------+-------------------------------------------------------------------------+
|user_id|merchant_abn|dollar_value      |order_id                            |order_datetime|merchant_name                 |tags                                                                     |consumer_id|state|postcode|gender|consumer_fraud_probability|merchant_fraud_probability|LOCALITY_NAME |list_SA2_CODE                    |list_ERP          |list_num_earners  |list_total_income_($M)|list_mean_inco

In [5]:
result_df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_fraud_probability: double (nullable = true)
 |-- merchant_fraud_probability: double (nullable = true)
 |-- LOCALITY_NAME: string (nullable = true)
 |-- list_SA2_CODE: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_ERP: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_num_earners: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_total_income_($M): array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- list_m

In [18]:
result_df = result_df.withColumn("list_fixed_tags", F.col("fixed_tags").cast("array"))

Py4JJavaError: An error occurred while calling o177.cast.
: scala.MatchError: array (of class java.lang.String)
	at org.apache.spark.sql.errors.QueryParsingErrors$.nestedTypeMissingElementTypeError(QueryParsingErrors.scala:286)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.$anonfun$visitPrimitiveDataType$1(DataTypeAstBuilder.scala:88)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin(SparkParserUtils.scala:125)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils.withOrigin$(SparkParserUtils.scala:115)
	at org.apache.spark.sql.catalyst.util.SparkParserUtils$.withOrigin(SparkParserUtils.scala:156)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:60)
	at org.apache.spark.sql.catalyst.parser.DataTypeAstBuilder.visitPrimitiveDataType(DataTypeAstBuilder.scala:32)
	at org.apache.spark.sql.catalyst.parser.SqlBaseParser$PrimitiveDataTypeContext.accept(SqlBaseParser.java:21753)
	at org.apache.spark.sql.

In [4]:
result_df = result_df \
        .withColumn('description', F.col('fixed_tags')[0])\
        .withColumn('revenue_bands', F.col('fixed_tags')[1])\
        .withColumn('take_rate', F.col('fixed_tags')[2])
result_df.show(5, truncate=False)

AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "fixed_tags". Need a complex type [STRUCT, ARRAY, MAP] but got "STRING".

In [9]:
result_df.write.mode("overwrite").parquet("../data/merged/merged_transactions_with_tags.parquet")